# Tutorial 2

In tutorial 1, we used the CLI and generated notebook to create data source, expectation suites and check points. In this tutorial, we will try to create a notebook by ourselves and do all the actions with one single notebook.


In [ ]:
%matplotlib inline

from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.data_context.types.resource_identifiers import (
    ExpectationSuiteIdentifier,
)
from great_expectations.exceptions import DataContextError
import great_expectations as ge
from great_expectations.cli.datasource import sanitize_yaml_and_save_datasource, check_if_datasource_name_exists

In [4]:
# get the project context
context = ge.get_context()

# 1. Create a new pandas Datasource
Here we create a file based datasource

In [6]:
# you can name it as you want
datasource_name = "pengfei_test"

my_datasource_yaml = f"""
name: {datasource_name}
class_name: Datasource
execution_engine:
  class_name: PandasExecutionEngine
data_connectors:
  default_inferred_data_connector_name:
    class_name: InferredAssetFilesystemDataConnector
    base_directory: ../../data
    default_regex:
      group_names:
        - data_asset_name
      pattern: (.*)
  default_runtime_data_connector_name:
    class_name: RuntimeDataConnector
    batch_identifiers:
      - default_identifier_name
"""
print(my_datasource_yaml)


name: pengfei_test
class_name: Datasource
execution_engine:
  class_name: PandasExecutionEngine
data_connectors:
  default_inferred_data_connector_name:
    class_name: InferredAssetFilesystemDataConnector
    base_directory: ../../data
    default_regex:
      group_names:
        - data_asset_name
      pattern: (.*)
  default_runtime_data_connector_name:
    class_name: RuntimeDataConnector
    batch_identifiers:
      - default_identifier_name



## 1.1 Test if your data source configuration is valid

In [7]:
context.test_yaml_config(yaml_config=my_datasource_yaml)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_inferred_data_connector_name : InferredAssetFilesystemDataConnector

	Available data_asset_names (2 of 2):
		adult.csv (1 of 1): ['adult.csv']
		adult_with_duplicates.csv (1 of 1): ['adult_with_duplicates.csv']

	Unmatched data_references (0 of 0):[]

	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



## 1.2 save the datasource

In [8]:
sanitize_yaml_and_save_datasource(context, my_datasource_yaml, overwrite_existing=False)


In [ ]:
## 1.3 List existing datasource

In [9]:
context.list_datasources()

[{'execution_engine': {'class_name': 'PandasExecutionEngine',
   'module_name': 'great_expectations.execution_engine'},
  'class_name': 'Datasource',
  'module_name': 'great_expectations.datasource',
  'data_connectors': {'default_inferred_data_connector_name': {'base_directory': '../../data',
    'module_name': 'great_expectations.datasource.data_connector',
    'default_regex': {'group_names': ['data_asset_name'], 'pattern': '(.*)'},
    'class_name': 'InferredAssetFilesystemDataConnector'},
   'default_runtime_data_connector_name': {'batch_identifiers': ['default_identifier_name'],
    'module_name': 'great_expectations.datasource.data_connector',
    'class_name': 'RuntimeDataConnector'}},
  'name': 'census_income_validation'},
 {'execution_engine': {'class_name': 'PandasExecutionEngine',
   'module_name': 'great_expectations.execution_engine'},
  'class_name': 'Datasource',
  'module_name': 'great_expectations.datasource',
  'data_connectors': {'default_inferred_data_connector_nam

# 2. Create or Edit expectation suits

Below code takes an **expectation suit name** as input, if it exists in the project, it returns the e

In [12]:
expectation_suite_name = "pengfei.test1"


try:
    suite = context.get_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(
        f'Loaded ExpectationSuite "{suite.expectation_suite_name}" containing {len(suite.expectations)} expectations.'
    )
except DataContextError:
    suite = context.create_expectation_suite(
        expectation_suite_name=expectation_suite_name
    )
    print(f'Created ExpectationSuite "{suite.expectation_suite_name}".')

Created ExpectationSuite "pengfei.test1".


# 2.1 Add expectations to the expectation suite

After getting the instance of the expectation suite, we can and need to add expectations(validation rules) to the suite.
Below is an example, we add a rule that verify column age can not have null values.

For now, if the expectation suite already contains expectations(validation rules), we can't remove them by using the suite object. We can only add new rules.



In [13]:
# define a validation rule, it's a dict fill with key/value
expectation_configuration = ExpectationConfiguration(
    **{
        "meta": {},
        "expectation_type": "expect_column_values_to_not_be_null",
        "kwargs": {"column": "age"},
    }
)

# add the rule to the suite
suite.add_expectation(expectation_configuration=expectation_configuration)

{"kwargs": {"column": "age"}, "meta": {}, "expectation_type": "expect_column_values_to_not_be_null"}

# 2.2 Save the expectation suite to the project context

After you save the expectation, it will create a new directory in the folder "expectations". The name of the created directory is the first part of the expectation suite name (i.e. pengfei). The json file in the directory (i.e. test1.json) is the actual file that contains all the expectations (validation rules)

```text
├── great_expectations
│         ├── checkpoints
│         ├── expectations
│                   ├── census_income_expectation_suite
│                   └── pengfei
|                         |__test1.json

│         ├── plugins
│         └── uncommitted
└── README.md

```

In [14]:
context.save_expectation_suite(expectation_suite=suite, expectation_suite_name=expectation_suite_name)

# we can get an identifier by using the name of expectation suite
suite_identifier = ExpectationSuiteIdentifier(expectation_suite_name=expectation_suite_name)

# use the below command will use the config in the expectation folders to generate a web page that contains the
# information of the newly created expectation suite
context.build_data_docs(resource_identifiers=[suite_identifier])

# open the web page in a browser
context.open_data_docs(resource_identifier=suite_identifier)

You should see the below page in your browser

![Expectation_ui](../images/expectations_validation_rule.png)
